# GNN Transport Mode Recommendation Model

This notebook builds and trains a Graph Neural Network to recommend the best transport mode (train, bus, luxury bus, or ride-hailing) for a given origin → destination query.

**Key Idea**: A transport network graph where:
- **Nodes**: Locations (Colombo, Anuradhapura, etc.)
- **Edges**: Services connecting locations with features (mode, duration, fare, comfort)
- **Task**: Predict which service is best (minimize delay) given departure time, date, poya status, etc.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import os
from datetime import datetime

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

print("Libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 1. Load Data

Load the synthetic transport network data from CSV files.

In [ ]:
# Load all CSV files
data_dir = '../data'

nodes_df = pd.read_csv(f'{data_dir}/nodes.csv')
services_df = pd.read_csv(f'{data_dir}/services.csv')
edges_df = pd.read_csv(f'{data_dir}/edges.csv')
calendar_df = pd.read_csv(f'{data_dir}/calendar.csv')
timetables_df = pd.read_csv(f'{data_dir}/timetables.csv')
performance_df = pd.read_csv(f'{data_dir}/performance_history.csv')
conditions_df = pd.read_csv(f'{data_dir}/service_conditions.csv')

print("📊 Data loaded successfully!")
print(f"Nodes: {len(nodes_df)}, Services: {len(services_df)}, Edges: {len(edges_df)}")
print(f"Performance records: {len(performance_df)}, Conditions: {len(conditions_df)}")
print(f"\nNode types: {nodes_df['type'].unique()}")
print(f"Transport modes: {services_df['mode'].unique()}")

## 2. Prepare Graph Data

Create graph structure for GNN:
- **Nodes**: Locations with features (region, type)
- **Edges**: Services connecting locations with features (mode, distance, duration, fare)
- **Labels**: User ratings from performance history

In [ ]:
# Encode categorical features
region_encoder = LabelEncoder()
type_encoder = LabelEncoder()
mode_encoder = LabelEncoder()

nodes_df['region_encoded'] = region_encoder.fit_transform(nodes_df['region'])
nodes_df['type_encoded'] = type_encoder.fit_transform(nodes_df['type'])
services_df['mode_encoded'] = mode_encoder.fit_transform(services_df['mode'])

# Create node features: [latitude, longitude, region_encoded, type_encoded]
node_features = torch.tensor(
    nodes_df[['latitude', 'longitude', 'region_encoded', 'type_encoded']].values,
    dtype=torch.float
)

# Normalize node features
scaler = StandardScaler()
node_features = torch.tensor(scaler.fit_transform(node_features), dtype=torch.float)

print(f"Node features shape: {node_features.shape}")
print(f"Node feature example:\n{node_features[0]}")

In [ ]:
# Build edge index and edge features
# Edge index: [2, num_edges] where each column is [source_node, target_node]
edge_list = []
edge_features_list = []

for _, edge in edges_df.iterrows():
    # Find node indices (location_id starts from 1, we need 0-indexed)
    src = edge['origin_id'] - 1
    dst = edge['destination_id'] - 1
    
    edge_list.append([src, dst])
    
    # Get service details
    service = services_df[services_df['service_id'] == edge['service_id']].iloc[0]
    
    # Edge features: [mode_encoded, distance, duration, fare]
    edge_features_list.append([
        service['mode_encoded'],
        service['distance_km'],
        service['base_duration_min'],
        service['base_fare']
    ])

edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor(edge_features_list, dtype=torch.float)

# Normalize edge features
edge_attr = torch.tensor(scaler.fit_transform(edge_attr), dtype=torch.float)

print(f"Edge index shape: {edge_index.shape}")
print(f"Edge attributes shape: {edge_attr.shape}")
print(f"Number of edges: {edge_index.shape[1]}")

In [ ]:
# Prepare training data from performance history
# Map service_id to edge index for labels
service_to_edge = {}
for idx, edge in edges_df.iterrows():
    service_to_edge[edge['service_id']] = idx

# Create training samples with labels (user ratings)
train_edges = []
train_labels = []

for _, perf in performance_df.iterrows():
    service_id = perf['service_id']
    if service_id in service_to_edge:
        edge_idx = service_to_edge[service_id]
        train_edges.append(edge_idx)
        # Normalize rating to 0-1 (original 1-5)
        train_labels.append((perf['user_rating'] - 1) / 4.0)

train_edges = torch.tensor(train_edges, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.float)

print(f"Training samples: {len(train_edges)}")
print(f"Label distribution (0-1 normalized):")
print(f"  Min: {train_labels.min():.2f}, Max: {train_labels.max():.2f}, Mean: {train_labels.mean():.2f}")

## 3. Define GNN Model

Graph Convolutional Network that learns node embeddings and predicts edge quality (service ratings).

In [ ]:
class TransportGNN(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim=64, output_dim=1):
        super(TransportGNN, self).__init__()
        
        # Graph convolution layers
        self.conv1 = GCNConv(node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        
        # Edge prediction layers
        self.edge_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2 + edge_features, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, output_dim),
            nn.Sigmoid()  # Output 0-1 rating
        )
        
    def forward(self, x, edge_index, edge_attr, target_edges):
        # Message passing to learn node embeddings
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = torch.relu(self.conv3(x, edge_index))
        
        # Predict ratings for target edges
        src_nodes = edge_index[0, target_edges]
        dst_nodes = edge_index[1, target_edges]
        
        # Concatenate source, destination embeddings and edge features
        edge_embeddings = torch.cat([
            x[src_nodes],
            x[dst_nodes],
            edge_attr[target_edges]
        ], dim=1)
        
        # Predict rating
        predictions = self.edge_mlp(edge_embeddings).squeeze()
        
        return predictions

# Initialize model
model = TransportGNN(
    node_features=node_features.shape[1],
    edge_features=edge_attr.shape[1],
    hidden_dim=64
)

print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
print(model)

## 4. Train the Model

Train GNN to predict service ratings based on historical user feedback.

In [ ]:
# Split data into train/validation
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(
    range(len(train_edges)), 
    test_size=0.2, 
    random_state=42
)

train_edges_split = train_edges[train_idx]
train_labels_split = train_labels[train_idx]
val_edges_split = train_edges[val_idx]
val_labels_split = train_labels[val_idx]

print(f"Training samples: {len(train_edges_split)}")
print(f"Validation samples: {len(val_edges_split)}")

In [ ]:
# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
criterion = nn.MSELoss()

# Training loop
num_epochs = 100
train_losses = []
val_losses = []

print("Starting training...")
for epoch in range(num_epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    
    predictions = model(node_features, edge_index, edge_attr, train_edges_split)
    loss = criterion(predictions, train_labels_split)
    
    loss.backward()
    optimizer.step()
    
    train_losses.append(loss.item())
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_predictions = model(node_features, edge_index, edge_attr, val_edges_split)
        val_loss = criterion(val_predictions, val_labels_split)
        val_losses.append(val_loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

print("\n✅ Training complete!")

## 5. Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('GNN Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final Training Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")

## 6. Evaluate Model Predictions

Test the model on all services and see how it ranks different transport modes.

In [ ]:
# Predict ratings for all edges
model.eval()
with torch.no_grad():
    all_edge_indices = torch.arange(len(edges_df))
    all_predictions = model(node_features, edge_index, edge_attr, all_edge_indices)
    
# Convert predictions back to 1-5 scale
predicted_ratings = (all_predictions.numpy() * 4) + 1

# Add predictions to edges dataframe
edges_df['predicted_rating'] = predicted_ratings

# Show results sorted by predicted rating
results = edges_df.merge(services_df, on='service_id')
results = results[['service_id', 'mode', 'operator', 'base_duration_min', 'base_fare', 'predicted_rating']]
results = results.sort_values('predicted_rating', ascending=False)

print("🎯 Model Predictions (Best to Worst):\n")
print(results.to_string(index=False))

# Group by mode
print("\n📊 Average Rating by Transport Mode:")
mode_ratings = results.groupby('mode')['predicted_rating'].mean().sort_values(ascending=False)
for mode, rating in mode_ratings.items():
    print(f"  {mode.capitalize():15s}: {rating:.2f}/5.0")

## 7. Save Trained Model

In [ ]:
# Save model
model_path = '../model/transport_gnn_model.pth'
os.makedirs('../model', exist_ok=True)

torch.save({
    'model_state_dict': model.state_dict(),
    'node_features': node_features,
    'edge_index': edge_index,
    'edge_attr': edge_attr,
    'region_encoder': region_encoder,
    'type_encoder': type_encoder,
    'mode_encoder': mode_encoder,
    'scaler': scaler
}, model_path)

print(f"✅ Model saved to {model_path}")
print(f"Model size: {os.path.getsize(model_path) / 1024:.2f} KB")

## 8. Test Recommendation System

Simulate a user query: "Colombo → Anuradhapura" and recommend best transport mode.

In [ ]:
def recommend_transport(origin_name, destination_name):
    """
    Recommend best transport service for a given origin-destination pair.
    """
    # Find origin and destination IDs
    origin_node = nodes_df[nodes_df['name'].str.contains(origin_name, case=False)]
    dest_node = nodes_df[nodes_df['name'].str.contains(destination_name, case=False)]
    
    if len(origin_node) == 0 or len(dest_node) == 0:
        print(f"❌ Could not find nodes for {origin_name} → {destination_name}")
        return
    
    origin_id = origin_node.iloc[0]['location_id']
    dest_id = dest_node.iloc[0]['location_id']
    
    # Find services connecting these locations
    relevant_services = services_df[
        (services_df['origin_id'] == origin_id) & 
        (services_df['destination_id'] == dest_id)
    ]
    
    if len(relevant_services) == 0:
        print(f"❌ No direct services found for {origin_name} → {destination_name}")
        return
    
    # Get predictions for these services
    service_recommendations = []
    for _, service in relevant_services.iterrows():
        edge = edges_df[edges_df['service_id'] == service['service_id']]
        if len(edge) > 0:
            rating = edge.iloc[0]['predicted_rating']
            service_recommendations.append({
                'service_id': service['service_id'],
                'mode': service['mode'],
                'operator': service['operator'],
                'duration': service['base_duration_min'],
                'fare': service['base_fare'],
                'rating': rating
            })
    
    # Sort by rating
    service_recommendations = sorted(service_recommendations, key=lambda x: x['rating'], reverse=True)
    
    print(f"\n🎯 Recommendations for {origin_name} → {destination_name}:\n")
    print(f"{'Rank':<5} {'Mode':<12} {'Operator':<12} {'Duration':<10} {'Fare':<8} {'Rating':<8}")
    print("=" * 70)
    
    for i, rec in enumerate(service_recommendations, 1):
        duration_str = f"{rec['duration']}min"
        fare_str = f"Rs.{rec['fare']}"
        rating_str = f"{rec['rating']:.2f}/5.0"
        
        emoji = "🥇" if i == 1 else ("🥈" if i == 2 else "🥉" if i == 3 else "  ")
        print(f"{emoji} {i:<3} {rec['mode']:<12} {rec['operator']:<12} {duration_str:<10} {fare_str:<8} {rating_str:<8}")
    
    best = service_recommendations[0]
    print(f"\n✨ Best recommendation: {best['mode'].upper()} ({best['operator']}) - {best['rating']:.2f}/5.0")
    return service_recommendations

# Test the recommendation system
recommend_transport("Colombo", "Anuradhapura")

## Summary

✅ **GNN Transport Recommendation System Complete!**

**What we built:**
1. Graph structure with 7 locations and 7 transport services
2. GNN model that learns from historical user ratings (200 samples)
3. Prediction system that ranks transport modes by quality
4. Saved model for deployment

**Key Features:**
- Considers: mode, distance, duration, fare
- Learns patterns from user feedback
- Provides ranked recommendations

**Dataset includes:**
- Service conditions (crowdedness by time/day)
- Calendar data (poya days, weekends)
- Timetable information

**Next Steps:**
- Integrate service_conditions.csv into predictions (time-of-day + poya day context)
- Add more locations and routes
- Collect real user feedback to improve model

## 9. DagsHub Integration

Track experiments, metrics, and models with DagsHub + MLflow.

In [ ]:
# Install dagshub and mlflow (run once)
# !pip install dagshub mlflow python-dotenv

import dagshub
import mlflow
import mlflow.pytorch
import os
from dotenv import load_dotenv

# Load environment variables from ml-services/.env
load_dotenv(dotenv_path='../../.env')

# Get DagsHub configuration from environment
DAGSHUB_REPO_OWNER = os.getenv("DAGSHUB_REPO_OWNER", "your-username")
DAGSHUB_REPO_NAME = os.getenv("DAGSHUB_REPO_NAME", "travion-research-project")

# Initialize DagsHub
dagshub.init(repo_owner=DAGSHUB_REPO_OWNER, repo_name=DAGSHUB_REPO_NAME, mlflow=True)

print("✅ DagsHub initialized!")
print(f"📊 Tracking URI: {mlflow.get_tracking_uri()}")
print(f"🔗 Repository: {DAGSHUB_REPO_OWNER}/{DAGSHUB_REPO_NAME}")
print(f"\n💡 Tip: Set credentials in ml-services/.env file")

### Re-train with MLflow Tracking

Now train the model again with full experiment tracking.

In [ ]:
# Re-initialize model for tracked training
model_tracked = TransportGNN(
    node_features=node_features.shape[1],
    edge_features=edge_attr.shape[1],
    hidden_dim=64
)

# Hyperparameters
params = {
    'learning_rate': 0.01,
    'weight_decay': 1e-5,
    'num_epochs': 100,
    'hidden_dim': 64,
    'batch_size': 'full',
    'optimizer': 'Adam',
    'loss_function': 'MSE',
    'train_size': len(train_edges_split),
    'val_size': len(val_edges_split),
    'num_nodes': len(nodes_df),
    'num_edges': len(edges_df)
}

# Start MLflow run with service tags
with mlflow.start_run(run_name="transport-gnn-v1"):
    # Set tags for organization
    mlflow.set_tag("service", "transport")
    mlflow.set_tag("model_type", "GNN")
    mlflow.set_tag("version", "v1")
    mlflow.set_tag("framework", "pytorch-geometric")
    
    # Log parameters
    mlflow.log_params(params)
    
    # Log dataset info
    mlflow.log_param("dataset_nodes", len(nodes_df))
    mlflow.log_param("dataset_services", len(services_df))
    mlflow.log_param("training_samples", len(performance_df))
    
    # Training setup
    optimizer_tracked = torch.optim.Adam(
        model_tracked.parameters(), 
        lr=params['learning_rate'], 
        weight_decay=params['weight_decay']
    )
    criterion_tracked = nn.MSELoss()
    
    print("🚀 Starting tracked training...")
    
    for epoch in range(params['num_epochs']):
        # Training
        model_tracked.train()
        optimizer_tracked.zero_grad()
        
        predictions = model_tracked(node_features, edge_index, edge_attr, train_edges_split)
        loss = criterion_tracked(predictions, train_labels_split)
        
        loss.backward()
        optimizer_tracked.step()
        
        # Validation
        model_tracked.eval()
        with torch.no_grad():
            val_predictions = model_tracked(node_features, edge_index, edge_attr, val_edges_split)
            val_loss = criterion_tracked(val_predictions, val_labels_split)
        
        # Log metrics every epoch
        mlflow.log_metric("train_loss", loss.item(), step=epoch)
        mlflow.log_metric("val_loss", val_loss.item(), step=epoch)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{params['num_epochs']} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")
    
    # Final metrics
    final_train_loss = loss.item()
    final_val_loss = val_loss.item()
    
    mlflow.log_metric("final_train_loss", final_train_loss)
    mlflow.log_metric("final_val_loss", final_val_loss)
    
    # Log model
    mlflow.pytorch.log_model(model_tracked, "model")
    
    # Save model artifacts
    model_artifacts = {
        'model_state_dict': model_tracked.state_dict(),
        'node_features': node_features,
        'edge_index': edge_index,
        'edge_attr': edge_attr,
        'region_encoder': region_encoder,
        'type_encoder': type_encoder,
        'mode_encoder': mode_encoder,
        'scaler': scaler
    }
    
    artifact_path = '../model/transport_gnn_mlflow.pth'
    torch.save(model_artifacts, artifact_path)
    mlflow.log_artifact(artifact_path)
    
    # Log dataset files
    mlflow.log_artifact('../data/nodes.csv')
    mlflow.log_artifact('../data/services.csv')
    mlflow.log_artifact('../data/performance_history.csv')
    mlflow.log_artifact('../data/service_conditions.csv')
    
    # Get run info
    run_id = mlflow.active_run().info.run_id
    
    print("\n✅ Training complete and logged to DagsHub!")
    print(f"📊 Run ID: {run_id}")
    print(f"🔗 View: https://dagshub.com/{DAGSHUB_REPO_OWNER}/{DAGSHUB_REPO_NAME}/experiments")

### View Experiment Results

You can view your experiment results on DagsHub:
- **Experiments**: `https://dagshub.com/{DAGSHUB_REPO_OWNER}/{DAGSHUB_REPO_NAME}/experiments`
- **Metrics**: Training/validation loss curves
- **Models**: Versioned model artifacts
- **Datasets**: Data lineage tracking

**What's tracked:**
- ✅ Hyperparameters (learning rate, epochs, hidden dim)
- ✅ Training metrics (loss per epoch)
- ✅ Validation metrics
- ✅ Model artifacts (.pth file)
- ✅ Dataset files (CSVs)
- ✅ Git commit hash (automatic)